##### Copyright 2019 The TensorFlow Authors.

In [0]:
# @title dilisensikan di bawah Lisensi Apache, Versi 2.0 ("Lisensi");
# Anda tidak boleh menggunakan file ini kecuali sesuai dengan Lisensi.
# Anda dapat memperoleh salinan Lisensi di
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Kecuali diminta oleh hukum yang berlaku atau disetujui secara tertulis, perangkat lunak
# didistribusikan di bawah Lisensi didistribusikan pada DASAR "SEBAGAIMANA ADANYA",
# TANPA JAMINAN ATAU KETENTUAN APA PUN, APAPUN tersurat maupun tersirat.
# Lihat Lisensi untuk bahasa spesifik yang mengatur izin dan
# batasan di bawah Lisensi.

# TensorFlow 2 quickstart untuk tingkat lanjut

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/advanced"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Lihat di TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Jalankan di Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Lihat sumber kode di GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Unduh notebook</a>
  </td>
</table>

Ini adalah file notebook [Google Colaboratory] (https://colab.research.google.com/notebooks/welcome.ipynb). Program python akan dijalankan langsung dari browser — cara yang bagus untuk mempelajari dan menggunakan TensorFlow. Untuk mengikuti tutorial ini, jalankan notebook di Google Colab dengan mengklik tombol di bagian atas halaman ini.

1. Di halaman Colab, sambungkan ke runtime Python: Di menu sebelah kanan atas, pilih * CONNECT *.
2. Untuk menjalankan semua sel kode pada notebook: Pilih * Runtime *> * Run all *.

Download dan instal TensorFlow 2 dan impor TensorFlow ke dalam program Anda:

In [0]:
try:
  # %tensorflow_version hanya ada di Colab.
  %tensorflow_version 2.x
except Exception:
  pass


Impor TensorFlow ke dalam program Anda:

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

Siapkan [dataset MNIST] (http://yann.lecun.com/exdb/mnist/).

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Tambahkan dimensi chanel
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

Gunakan `tf.data` untuk mengelompokkan dan mengatur kembali dataset:

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Buat model `tf.keras` menggunakan Keras [model subclassing API] (https://www.tensorflow.org/guide/keras#model_subclassing):

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Buat sebuah contoh dari model
model = MyModel()

Pilih fungsi untuk mengoptimalkan dan fungsi untuk menilai loss dari hasil pelatihan:

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

Pilih metrik untuk mengukur loss dan keakuratan model. Metrik ini mengakumulasi nilai di atas epochs dan kemudian mencetak hasil secara keseluruhan.

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Gunakan `tf.GradientTape` untuk melatih model:

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [ ]:
Tes modelnya:

In [0]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [0]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Menghitung ulang metrik untuk epoch selanjutnya
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Penggolong gambar tersebut, sekarang dilatih untuk akurasi ~ 98% pada dataset ini. Untuk mempelajari lebih lanjut, baca [tutorial TensorFlow] (https://www.tensorflow.org/tutorials/).